In [1]:
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.metrics import accuracy_score
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from keras.applications import MobileNet
from keras.layers import GlobalAveragePooling2D, Dense
from keras.models import Model
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
dataset_path = "/content/drive/MyDrive/CEG/SEM 3/Final Year Mtech Project/Simple CNN/dataset"
output_path = "/content/drive/MyDrive/CEG/SEM 3/Final Year Mtech Project/Simple CNN/preprocessed_images"

# if not os.path.exists(output_path):
#     os.makedirs(output_path)

In [ ]:
# for species in os.listdir(dataset_path):
#     species_path = os.path.join(dataset_path, species)
#     species_output_path = os.path.join(output_path, species)

#     if not os.path.exists(species_output_path):
#         os.makedirs(species_output_path)

#     images = os.listdir(species_path)

#     for image_name in images:
#         image_path = os.path.join(species_path, image_name)
#         img = cv2.imread(image_path)

#         if img is not None:
#             img = cv2.resize(img, (128, 128))

#             img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

#             output_file_path = os.path.join(species_output_path, image_name)
#             Image.fromarray(img).save(output_file_path)

In [ ]:
# for species in os.listdir(output_path):
#     species_output_path = os.path.join(output_path, species)
#     images = os.listdir(species_output_path)

#     plt.figure(figsize=(10, 4))

#     original_image = cv2.imread(os.path.join(dataset_path, species, images[0]))
#     plt.subplot(1, 2, 1)
#     plt.imshow(cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB))
#     plt.title(f"{species} - Original Image")
#     plt.axis('off')

#     preprocessed_image = cv2.imread(os.path.join(species_output_path, images[0]), cv2.IMREAD_GRAYSCALE)
#     plt.subplot(1, 2, 2)
#     plt.imshow(preprocessed_image, cmap='gray')
#     plt.title(f"{species} - Preprocessed Image")
#     plt.axis('off')

#     plt.tight_layout()
#     plt.show()


In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.callbacks import ModelCheckpoint





In [5]:
data_dir = output_path

In [6]:
image_paths = []
labels = []

for class_label, class_name in enumerate(os.listdir(data_dir)):
    class_path = os.path.join(data_dir, class_name)
    for image_name in os.listdir(class_path):
        image_paths.append(os.path.join(class_path, image_name))
        labels.append(class_label)

In [7]:
x_train, x_test, y_train, y_test = train_test_split(image_paths, labels, test_size=0.2, random_state=42)


In [8]:
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(128, 128, 3))
model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(8, activation='softmax'))
model.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])


9406464/9406464 [==============================] - 0s 0us/step


In [9]:
x_train_images = []
x_test_images = []

for img_path in x_train:
    img = image.load_img(img_path, target_size=(128, 128))
    img = image.img_to_array(img)
    img = preprocess_input(img)
    x_train_images.append(img)
    print(img_path)

for img_path in x_test:
    img = image.load_img(img_path, target_size=(128, 128))
    img = image.img_to_array(img)
    img = preprocess_input(img)
    x_test_images.append(img)
    print(img_path)

x_train_images = np.array(x_train_images)
x_test_images = np.array(x_test_images)

Streaming output truncated to the last 5000 lines.
/content/drive/MyDrive/CEG/SEM 3/Final Year Mtech Project/Simple CNN/preprocessed_images/Abrus precatorius/2_3_Image_29.jpg
/content/drive/MyDrive/CEG/SEM 3/Final Year Mtech Project/Simple CNN/preprocessed_images/Iris species/4_Image_37.jpg
/content/drive/MyDrive/CEG/SEM 3/Final Year Mtech Project/Simple CNN/preprocessed_images/Carex bronze/3_0_Image_35.jpg
/content/drive/MyDrive/CEG/SEM 3/Final Year Mtech Project/Simple CNN/preprocessed_images/Tamarindus indica red/2_0_Image_15.jpg
/content/drive/MyDrive/CEG/SEM 3/Final Year Mtech Project/Simple CNN/preprocessed_images/Zephyranthes grandiflora/4_2_Image_14.jpg
/content/drive/MyDrive/CEG/SEM 3/Final Year Mtech Project/Simple CNN/preprocessed_images/Tamarindus indica red/3_3_Image_46.jpg
/content/drive/MyDrive/CEG/SEM 3/Final Year Mtech Project/Simple CNN/preprocessed_images/Zephyranthes grandiflora/1_Image_26.jpg
/content/drive/MyDrive/CEG/SEM 3/Final Year Mtech Project/Simple CNN/prep

In [10]:
# Convert target labels to NumPy arrays
y_train = np.array(y_train)
y_test = np.array(y_test)

In [11]:
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    zoom_range=0.1
)

datagen.fit(x_train_images)

In [12]:
model_checkpoint = ModelCheckpoint("Mobinet.h5", save_best_only=True, monitor="val_accuracy", mode="max", verbose=1)


In [19]:
model.fit(datagen.flow(x_train_images, y_train, batch_size=32),
          epochs=5,
          validation_data=(x_test_images, y_test),callbacks=[model_checkpoint])

Epoch 1/5
251/251 [==============================] - ETA: 0s - loss: 0.0467 - accuracy: 0.9870
Epoch 1: val_accuracy did not improve from 0.92818
251/251 [==============================] - 41s 163ms/step - loss: 0.0467 - accuracy: 0.9870 - val_loss: 1.0950 - val_accuracy: 0.8623
Epoch 2/5
251/251 [==============================] - ETA: 0s - loss: 0.0446 - accuracy: 0.9859
Epoch 2: val_accuracy did not improve from 0.92818
251/251 [==============================] - 40s 160ms/step - loss: 0.0446 - accuracy: 0.9859 - val_loss: 3.7512 - val_accuracy: 0.7117
Epoch 3/5
251/251 [==============================] - ETA: 0s - loss: 0.0429 - accuracy: 0.9873
Epoch 3: val_accuracy did not improve from 0.92818
251/251 [==============================] - 40s 159ms/step - loss: 0.0429 - accuracy: 0.9873 - val_loss: 2.2209 - val_accuracy: 0.6968
Epoch 4/5
251/251 [==============================] - ETA: 0s - loss: 0.0628 - accuracy: 0.9819
Epoch 4: val_accuracy did not improve from 0.92818
251/251 [=====

In [20]:
predictions = model.predict(x_test_images)
predicted_labels = np.argmax(predictions, axis=1)
results_df = pd.DataFrame({'Actual': y_test, 'Predicted': predicted_labels})
print(results_df)

63/63 [==============================] - 1s 19ms/step
      Actual  Predicted
0          4          6
1          4          4
2          3          2
3          0          0
4          7          7
...      ...        ...
2000       4          4
2001       1          1
2002       2          2
2003       3          3
2004       2          2

[2005 rows x 2 columns]


In [22]:
y_test = np.array(y_test)

# Evaluation
loss, accuracy = model.evaluate(x_test_images, y_test)
print(f"Accuracy on the test set: {accuracy * 100:.2f}%")

63/63 [==============================] - 1s 19ms/step - loss: 4.0025 - accuracy: 0.7302
Accuracy on the test set: 73.02%


# VGG16 Approach

In [ ]:
import os
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing import image

In [ ]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

58889256/58889256 [==============================] - 0s 0us/step


In [ ]:
model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(8, activation='softmax'))  # Adjust the number of classes (10) to your use case

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model_checkpoint = ModelCheckpoint("vgg16.h5", save_best_only=True, monitor="val_accuracy", mode="max", verbose=1)


In [ ]:
model.fit(datagen.flow(x_train_images, y_train, batch_size=32),
          epochs=5,
          validation_data=(x_test_images, y_test), callbacks=[model_checkpoint])

Epoch 1/5
251/251 [==============================] - ETA: 0s - loss: 0.4868 - accuracy: 0.8207
Epoch 1: val_accuracy did not improve from 0.83840
251/251 [==============================] - 52s 208ms/step - loss: 0.4868 - accuracy: 0.8207 - val_loss: 0.4887 - val_accuracy: 0.8274
Epoch 2/5
251/251 [==============================] - ETA: 0s - loss: 0.4665 - accuracy: 0.8289
Epoch 2: val_accuracy did not improve from 0.83840
251/251 [==============================] - 48s 192ms/step - loss: 0.4665 - accuracy: 0.8289 - val_loss: 0.5300 - val_accuracy: 0.8294
Epoch 3/5
251/251 [==============================] - ETA: 0s - loss: 0.3880 - accuracy: 0.8548
Epoch 3: val_accuracy improved from 0.83840 to 0.87032, saving model to vgg16.h5
251/251 [==============================] - 54s 216ms/step - loss: 0.3880 - accuracy: 0.8548 - val_loss: 0.4087 - val_accuracy: 0.8703
Epoch 4/5
251/251 [==============================] - ETA: 0s - loss: 0.4368 - accuracy: 0.8461
Epoch 4: val_accuracy improved from

In [ ]:
predictions = model.predict(x_test_images)
predicted_labels = np.argmax(predictions, axis=1)
results_df = pd.DataFrame({'Actual': y_test, 'Predicted': predicted_labels})
print(results_df)

63/63 [==============================] - 3s 39ms/step
      Actual  Predicted
0          4          0
1          4          4
2          3          3
3          0          0
4          7          7
...      ...        ...
2000       4          4
2001       1          1
2002       2          2
2003       3          3
2004       2          2

[2005 rows x 2 columns]


In [ ]:
y_test = np.array(y_test)

# Evaluation
loss, accuracy = model.evaluate(x_test_images, y_test)
print(f"Accuracy on the test set: {accuracy * 100:.2f}%")

63/63 [==============================] - 3s 40ms/step - loss: 0.3391 - accuracy: 0.8758
Accuracy on the test set: 87.58%


# Resnet

In [ ]:
import os
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing import image

In [ ]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

94765736/94765736 [==============================] - 1s 0us/step


In [ ]:
model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(8, activation='softmax'))

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model_checkpoint = ModelCheckpoint("best_model.h5", save_best_only=True, monitor="val_accuracy", mode="max", verbose=1)

In [ ]:
model.fit(datagen.flow(x_train_images, y_train, batch_size=32),
          epochs=5,
          validation_data=(x_test_images, y_test), callbacks=[model_checkpoint] )

Epoch 1/5
251/251 [==============================] - ETA: 0s - loss: 0.1131 - accuracy: 0.9646
Epoch 1: val_accuracy did not improve from 0.94963
251/251 [==============================] - 48s 191ms/step - loss: 0.1131 - accuracy: 0.9646 - val_loss: 4.5259 - val_accuracy: 0.5835
Epoch 2/5
251/251 [==============================] - ETA: 0s - loss: 0.1037 - accuracy: 0.9658
Epoch 2: val_accuracy did not improve from 0.94963
251/251 [==============================] - 50s 199ms/step - loss: 0.1037 - accuracy: 0.9658 - val_loss: 0.5925 - val_accuracy: 0.8643
Epoch 3/5
251/251 [==============================] - ETA: 0s - loss: 0.0968 - accuracy: 0.9714
Epoch 3: val_accuracy improved from 0.94963 to 0.95262, saving model to best_model.h5
251/251 [==============================] - 58s 233ms/step - loss: 0.0968 - accuracy: 0.9714 - val_loss: 0.1706 - val_accuracy: 0.9526
Epoch 4/5
251/251 [==============================] - ETA: 0s - loss: 0.0810 - accuracy: 0.9754
Epoch 4: val_accuracy did not 

In [ ]:
predictions = model.predict(x_test_images)
predicted_labels = np.argmax(predictions, axis=1)
results_df = pd.DataFrame({'Actual': y_test, 'Predicted': predicted_labels})
print(results_df)

63/63 [==============================] - 3s 36ms/step
      Actual  Predicted
0          4          6
1          4          4
2          3          3
3          0          0
4          7          7
...      ...        ...
2000       4          4
2001       1          6
2002       2          2
2003       3          3
2004       2          2

[2005 rows x 2 columns]


In [ ]:
y_test = np.array(y_test)

# Evaluation
loss, accuracy = model.evaluate(x_test_images, y_test)
print(f"Accuracy on the test set: {accuracy * 100:.2f}%")

63/63 [==============================] - 2s 38ms/step - loss: 0.6508 - accuracy: 0.8524
Accuracy on the test set: 85.24%


# different Approach

In [ ]:
# x = base_model.output
# x = GlobalAveragePooling2D()(x)
# x = Dense(256, activation='relu')(x)
# predictions = Dense(len(np.unique(labels)), activation='softmax')(x)

In [ ]:
# model = Model(inputs=base_model.input, outputs=predictions)


In [ ]:
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)
batch_size = 32  # Adjust as needed
train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(124, 124),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

validation_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(124, 124),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)


Found 502 images belonging to 10 classes.
Found 0 images belonging to 10 classes.


In [ ]:
batch_chunk_size = 52
epochs = 5

In [ ]:
datagen.fit(X_train)

ValueError: ignored